In [ ]:
import re
import pandas as pd
from bs4 import BeautifulSoup
from requests_futures.sessions import FuturesSession

In [ ]:
props = ['id', 'name', 'attr', 'rare', 'race', 'mode', 'skill1', 'skill2', 'skill3', 'charge']

In [ ]:
def get_craft_data(str_C):
    url = "https://tos.fandom.com/zh/api.php"
    params_querypage = {
        "action": "query",
        "prop": "revisions",
        "titles": "Template:C" + str_C,
        "rvslots" : "main",
        "rvprop": "content",
        "format": "json"
    }
    r = session.post(url, data=params_querypage)
    return r

In [ ]:
def get_craft_desc(r):
    try:
        page = list(r.json()['query']['pages'].values())[0]
        craft_info = page['revisions'][0]['slots']['main']['*']
        dict_craft = {}
        for prop in props:
            pattern_prop = prop + r'.*=.*\n'
            str_prop = re.search(pattern_prop, craft_info)
            if prop == 'name' and str_prop is None:
                return None
            if str_prop is not None:
                str_prop = str_prop[0]
                str_prop = get_desc_details(str_prop)
            else:
                str_prop = ''
            dict_craft[prop] = str_prop
        df_crafts = pd.Series(dict_craft, index = dict_craft.keys()) 
        return df_crafts
    except:
        return None

In [ ]:
def get_desc_details(str):
    pattern_detail = r'= (.*)\n'
    str = re.search(pattern_detail, str)
    if str is None:
        str = ''
    else:
        str = str.group(1)
    pattern_detail = r'(.*)\t'
    str2 = re.search(pattern_detail, str)
    if str2 is None:
        pass
    else:
        str = str2.group(1)
    return str

In [ ]:
df_crafts = pd.DataFrame()
urlquery = list()
session = FuturesSession(max_workers=20)
for C_n in range(1,500):
    if C_n < 10:
        str_C = str(C_n).zfill(2)
    else:
        str_C = str(C_n)
    urlquery.append( (str_C, get_craft_data(str_C)) )
for (str_C, res) in urlquery:
    res = res.result()
    df_craft = get_craft_desc(res)
    if df_craft is not None:
        df_crafts = df_crafts.append(df_craft, ignore_index=True)
display(df_crafts)
df_crafts.to_csv('crafts.csv', encoding='utf-8', index=False)

In [ ]:
# # show obtained data
# df2 = pd.read_csv('crafts.csv', encoding='utf-8')
# display(df2)